In [1]:
using Revise, Pkg
Pkg.activate("/home/louise/MSA/BpAlignGpu.jl")
using BpAlignGpu

  Activating project at `~/MSA/BpAlignGpu.jl`
┌ Info: Precompiling BpAlignGpu [5a3eb610-29b2-4cbe-8ba2-ea97f65fa95d]
└ @ Base loading.jl:1423


In [2]:
(q, N, L) = (21, 5, 5) 

(21, 5, 5)

In [3]:
function convert_T3_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, L)
    for i=1:L
        for n=1:N+2
            SCE_ar[n, i] = tmp[n,1,i]
            SCE_ar[N+2+n, i] = tmp[n,2,i]
        end
    end
    return SCE_ar
end
function convert_T5_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, 2N+4, L)
    for i=1:L
        for n1=1:N+2
            for n2=1:N+2
                SCE_ar[n1, n2, i] = tmp[n1, 1, n2, 1, i]
                SCE_ar[n1, N+2+n2, i] = tmp[n1, 1, n2, 2, i]
                SCE_ar[N+2+n1, n2, i] = tmp[n1, 2, n2, 1, i]
                SCE_ar[N+2+n1, N+2+n2, i] = tmp[n1, 2, n2, 2, i]
            end
        end
    end
    return SCE_ar
end
function convert_T6_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, 2N+4, L, L)
    for i=1:L
        for j=1:L
            for n1=1:N+2
                for n2=1:N+2
                    SCE_ar[n1, n2, i, j] = tmp[n1, 1, n2, 1, i, j]
                    SCE_ar[n1, N+2+n2, i, j] = tmp[n1, 1, n2, 2, i, j]
                    SCE_ar[N+2+n1, n2, i, j] = tmp[n1, 2, n2, 1, i, j]
                    SCE_ar[N+2+n1, N+2+n2, i, j] = tmp[n1, 2, n2, 2, i, j]
                end
            end
        end
    end
    return SCE_ar
end

convert_T6_xnTOy (generic function with 1 method)

In [4]:
T = Float32

Float32

In [5]:
using Random
header = "myseq"
myseq = randstring('A':'Z', N)
ctype=Symbol("amino")
seq = Seq(header, myseq, ctype)

myseq
NTJQP


In [6]:
muint = 1.5
muext = 1.2
lambda_o = ones(L)
lambda_e = ones(L)
H = rand(q,L)
J = rand(q,q,L,L)
J = J .+ permutedims(J, (2,1,4,3));

In [7]:
pm = ParamModel{T}(N, L, q, muint, muext, lambda_o, lambda_e, H, J)

ParamModel{Float32}[L=5 N=5 q=21]


In [ ]:
damp=T(0.0)
tol=T(1e-5)
tolnorm=T(1e-5)
tmax=10
upscheme=:sequential # :random or :sequential
lr=:sce  # :sce or :mf 
beta=T(1.0)
verbose=true
pa = ParamAlgo(damp, tol, tolnorm, tmax, upscheme, lr, beta, verbose)

In [ ]:
bpm = BPMessages(seq, pm)

In [ ]:
bpb = BPBeliefs(N, L)

In [ ]:
lrf = LongRangeFields(N, L)

In [ ]:
af = AllFields(bpm, bpb, lrf)

In [ ]:
tmp_F = Array(af.bpm.F)
SCE_F = convert_T3_xnTOy(tmp_F);

In [ ]:
tmp_hF = Array(af.bpm.hF)
SCE_hF = convert_T3_xnTOy(tmp_hF);

In [ ]:
tmp_hB = Array(af.bpm.hB)
SCE_hB = convert_T3_xnTOy(tmp_hB);

In [ ]:
tmp_B = Array(af.bpm.B)
SCE_B = convert_T3_xnTOy(tmp_B);

In [ ]:
tmp_g = Array(af.lrf.g)
SCE_g = convert_T5_xnTOy(tmp_g);

In [ ]:
tmp_f = Array(af.lrf.f)
SCE_f = convert_T3_xnTOy(tmp_f);

In [ ]:
tmp_Jseq = Array(af.bpm.Jseq)
elts_Jseq = convert_T6_xnTOy(tmp_Jseq);

In [ ]:
tmp_Hseq = Array(af.bpm.Hseq)
elts_Hseq = convert_T3_xnTOy(tmp_Hseq);

In [ ]:
damp = 0.0
beta = 1.0
tolnorm = 1e-5

In [ ]:
include("../src/SCE_update_messages.jl")
include("../src/SCE_update_shortrange_beliefs.jl")
include("../src/SCE_update_longrange.jl")

In [ ]:
BpAlignGpu.update_jointchain!(af, pm)

In [ ]:
i=4
res = SCE_update_joint_chain!(i, beta, tolnorm, SCE_F, SCE_B, SCE_g, lambda_o, lambda_e, N, elts_Jseq);

In [ ]:
af.bpb.joint_chain[:,1,:,2,i]

In [ ]:
rx1 = 1:N+2
rx2 = N+3:2N+4
res[rx1,rx2]

In [ ]:
BpAlignGpu.update_conditional_chain!(af, pa)

In [86]:
tmp_joint = Array(af.bpb.joint_chain)
SCE_joint = convert_T5_xnTOy(tmp_joint);

In [87]:
cond_chain = fill(0.0, 2N+4, 2N+4, L, L)
for i=1:L-1
    SCE_update_conditional_chain!(i, cond_chain, SCE_joint, tolnorm, N)
end

In [88]:
af.bpb.conditional[:,2,:,2,4,5]

7×7 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0  0.0       0.0        0.0        0.0         0.0
 0.0  0.0  0.252989  0.0318488  0.0198868  0.00862162  0.0
 0.0  0.0  0.0       0.339955   0.0981289  0.0395668   0.0
 0.0  0.0  0.0       0.0        0.539096   0.287536    0.0
 0.0  0.0  0.0       0.0        0.0        0.184589    0.0
 0.0  0.0  0.0       0.0        0.0        0.0         0.0
 0.0  0.0  0.0       0.0        0.0        0.0         0.0

In [89]:
cond_chain[rx2, rx2, 4, 5]

7×7 Matrix{Float64}:
 0.0  0.0  0.0       0.0        0.0        0.0         0.0
 0.0  0.0  0.252989  0.0318488  0.0198868  0.00862162  0.0
 0.0  0.0  0.0       0.339955   0.0981289  0.0395668   0.0
 0.0  0.0  0.0       0.0        0.539096   0.287536    0.0
 0.0  0.0  0.0       0.0        0.0        0.184589    0.0
 0.0  0.0  0.0       0.0        0.0        0.0         0.0
 0.0  0.0  0.0       0.0        0.0        0.0         0.0

In [90]:
cond_lr = copy(cond_chain);
pointer(cond_chain), pointer(cond_lr)

(Ptr{Float64} @0x000000000e334dc0, Ptr{Float64} @0x000000000f1c9f40)

In [91]:
SCE_update_conditional_lr!(cond_lr)

In [99]:
cond_lr[rx1, rx2, 4, 2]

7×7 Matrix{Float64}:
 0.0  0.0         0.0         0.0        0.0        0.0       0.0
 0.0  0.131432    0.0         0.0        0.0        0.0       0.0
 0.0  0.0427219   0.0949615   0.0        0.0        0.0       0.0
 0.0  0.03313     0.0701897   0.0633809  0.0        0.0       0.0
 0.0  0.0194413   0.00806216  0.0771436  0.0652614  0.0       0.0
 0.0  0.00860534  0.0277258   0.0650422  0.341302   0.470872  0.0
 0.0  0.37889     0.159053    0.188402   0.373228   0.529128  0.0

In [94]:
CR, CL = BpAlignGpu.update_conditional_all!(af, pm);

In [101]:
CL[:,1,:,2,4,2]

7×7 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0         0.0         0.0        0.0        0.0       0.0
 0.0  0.131432    0.0         0.0        0.0        0.0       0.0
 0.0  0.0427219   0.0949615   0.0        0.0        0.0       0.0
 0.0  0.03313     0.0701897   0.0633809  0.0        0.0       0.0
 0.0  0.0194413   0.00806216  0.0771436  0.0652614  0.0       0.0
 0.0  0.00860534  0.0277258   0.0650422  0.341302   0.470872  0.0
 0.0  0.37889     0.159053    0.188402   0.373228   0.529128  0.0

In [103]:
#res_forloop = BpAlignGpu.update_conditional_all_forloop!(af, pm);

In [94]:
#BpAlignGpu.update_hB!(af, pm, pa)

In [95]:
#i=1
#af.bpm.hB[:,:,i]

In [96]:
#SCE_update_F!(i, damp, beta, tolnorm, SCE_hF, SCE_f, muext, muint, N, elts_Hseq)
#SCE_update_B!(i, damp, beta, tolnorm, SCE_hB, SCE_f, muext, muint, N, L, elts_Hseq)
#SCE_update_hF!(i, SCE_F, damp, beta, tolnorm, SCE_g, pm.lambda_o, pm.lambda_e, N, elts_Jseq)
#SCE_update_hB!(i, damp, beta, tolnorm, SCE_B, SCE_g, pm.lambda_o, pm.lambda_e, N, elts_Jseq)

In [224]:
using CUDA
x = ones(T, L, L) |> cu

5×5 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0

In [225]:
xt = ones(T, L, L)

5×5 Matrix{Float32}:
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0

In [229]:
using Tullio
@tullio x[i, j] = (i<=L-2 && j>=i+2) ? (x[i,j-1] + x[j-1,j]) : x[i,j]
@tullio x[i, j] = (j<=L-2 && i>=j+2) ? (x[i,i-1] + x[i-1,j]) : x[i,j]


5×5 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1.0  1.0  2.0  3.0  4.0
 1.0  1.0  1.0  2.0  3.0
 2.0  1.0  1.0  1.0  2.0
 3.0  2.0  1.0  1.0  1.0
 4.0  3.0  2.0  1.0  1.0

In [232]:
for i=1:L-2
    for j=i+2:L
        xt[i,j] = xt[i,j-1]+xt[j-1,j]
    end
end    
for j=1:L-2
    for i=j+2:L
        xt[i,j] = xt[i,i-1]+xt[i-1,j]
    end
end
xt

5×5 Matrix{Float32}:
 1.0  1.0  2.0  3.0  4.0
 1.0  1.0  1.0  2.0  3.0
 2.0  1.0  1.0  1.0  2.0
 3.0  2.0  1.0  1.0  1.0
 4.0  3.0  2.0  1.0  1.0